In [1]:
import os, hashlib, datetime, json
print("✅ Lab 1 — Dependency & Model Integrity Check.\n")

REPORT = "lab1_integrity_report.md"
def write_header():
    with open(REPORT, "w", encoding="utf-8") as f:
        f.write("# Lab 1 — Dependency & Model Integrity Check\n")
        f.write(f"_Generated: {datetime.datetime.now().isoformat()}_\n\n")
def append_report(title, body):
    with open(REPORT, "a", encoding="utf-8") as f:
        f.write(f"## {title}\n{body}\n\n")
write_header()

# Create mock model files
ORIGINAL_MODEL_PATH = "model_original.bin"
TAMPERED_MODEL_PATH = "model_tampered.bin"
original_content = (
    "model_name: secure-llm\n"
    "version: 1.0.0\n"
    "vendor: trusted-ai-inc\n"
    "checksum_note: official release build\n"
)
tampered_content = (
    "model_name: secure-llm\n"
    "version: 1.0.0\n"
    "vendor: trusted-ai-inc\n"
    "checksum_note: unofficial debug build - logging enabled\n"
)
with open(ORIGINAL_MODEL_PATH, "w", encoding="utf-8") as f: f.write(original_content)
with open(TAMPERED_MODEL_PATH, "w", encoding="utf-8") as f: f.write(tampered_content)

def sha256_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            h.update(chunk)
    return h.hexdigest()

TRUSTED_HASH = sha256_file(ORIGINAL_MODEL_PATH)
print("Trusted hash for original model:\n", TRUSTED_HASH, "\n")
append_report("Trusted Hash", f"Trusted SHA-256 for model_original.bin:\n`{TRUSTED_HASH}`\n")

# Experiment A — Original model
hash_original = sha256_file(ORIGINAL_MODEL_PATH)
is_match_original = (hash_original == TRUSTED_HASH)
print("=== Experiment A — Original Model Integrity ===")
print("Computed hash:", hash_original)
print("Matches trusted hash?", is_match_original, "\n")
append_report("Experiment A — Original Model",
              f"Computed SHA-256: `{hash_original}`\nMatches trusted reference? **{is_match_original}**\n")

# Experiment B — Tampered model
hash_tampered = sha256_file(TAMPERED_MODEL_PATH)
is_match_tampered = (hash_tampered == TRUSTED_HASH)
print("=== Experiment B — Tampered Model Integrity ===")
print("Computed hash:", hash_tampered)
print("Matches trusted hash?", is_match_tampered, "\n")
append_report("Experiment B — Tampered Model",
              f"Computed SHA-256: `{hash_tampered}`\nMatches trusted reference? **{is_match_tampered}**\n")

# SBOM vs actual deps
SBOM = [
    {"name": "transformers", "version": "4.40.0", "source": "pypi.org"},
    {"name": "numpy",        "version": "1.26.0", "source": "pypi.org"},
    {"name": "fastapi",      "version": "0.110.0", "source": "pypi.org"},
]
ACTUAL_DEPS = [
    {"name": "transformers", "version": "4.38.0", "source": "pypi.org"},
    {"name": "numpy",        "version": "1.26.0", "source": "pypi.org"},
    {"name": "fastapi",      "version": "0.110.0", "source": "pypi.org"},
    {"name": "custom-llm-lib", "version": "0.1.0", "source": "unknown-github-user"},
]

def compare_sbom(expected, actual):
    issues = []
    expected_index = {d["name"]: d for d in expected}
    actual_index = {d["name"]: d for d in actual}
    for name, exp in expected_index.items():
        if name not in actual_index:
            issues.append({"type": "missing", "name": name, "expected": exp, "actual": None})
        else:
            act = actual_index[name]
            if act["version"] != exp["version"]:
                issues.append({"type": "version_mismatch",
                               "name": name,
                               "expected": exp["version"],
                               "actual": act["version"]})
    for name, act in actual_index.items():
        if name not in expected_index:
            issues.append({"type": "unexpected_dep",
                           "name": name,
                           "version": act["version"],
                           "source": act["source"]})
    return issues

issues = compare_sbom(SBOM, ACTUAL_DEPS)
print("=== SBOM Comparison Results ===")
print(json.dumps(issues, indent=2), "\n")
append_report("SBOM Comparison", f"SBOM issues:\n```json\n{json.dumps(issues, indent=2)}\n```\n")

summary = """
- Original model file matched the trusted hash → likely authentic.
- Tampered model file did NOT match the trusted hash → indicates modification.
- SBOM check found a version mismatch and an unexpected dependency from an unknown source.
"""
append_report("Summary", summary)
append_report("Reflection Questions",
"- Why is a matching hash important for model integrity?\n"
"- How can SBOMs help detect risky third-party components?\n"
"- What would you do if a vendor cannot provide a trusted hash or SBOM?\n")

print("🎯 Lab 1 complete. Download lab1_integrity_report.md from the Files pane.")

✅ Lab 1 — Dependency & Model Integrity Check.

Trusted hash for original model:
 f805e78b6212b8cda47fccfc46905e4b7ef6e48160fa49cd588617650978a5a9 

=== Experiment A — Original Model Integrity ===
Computed hash: f805e78b6212b8cda47fccfc46905e4b7ef6e48160fa49cd588617650978a5a9
Matches trusted hash? True 

=== Experiment B — Tampered Model Integrity ===
Computed hash: bdb7f997a044879f3a7f3024150a94aa4f4c513c9f824093d3a8211ac470e610
Matches trusted hash? False 

=== SBOM Comparison Results ===
[
  {
    "type": "version_mismatch",
    "name": "transformers",
    "expected": "4.40.0",
    "actual": "4.38.0"
  },
  {
    "type": "unexpected_dep",
    "name": "custom-llm-lib",
    "version": "0.1.0",
    "source": "unknown-github-user"
  }
] 

🎯 Lab 1 complete. Download lab1_integrity_report.md from the Files pane.
